1028
자동 인코더 만들기

In [171]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, UpSampling2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

In [172]:
(X_train, _), (X_test, _) = tf.keras.datasets.fashion_mnist.load_data()

In [173]:
X_train = X_train.reshape(60000, 28*28)# Flatten 시킴
X_test = X_test.reshape(10000, 28*28) 

In [174]:
X_train = X_train/255 # Flatten 시킴
X_test = X_test /255 

In [175]:
class MyModel(tf.keras.models.Model): 
    def __int__(self, dim): 
        super(MyModel, self).__init__() 
        self.dim = dim 
        self.encoder = tf.keras.layers.Dense(dim)
        self.decoder = tf.keras.layers.Dense(28*28, activation='sigmoid')   #activation은 0~1사이의 값으로 범위결정하는 애.
    def call(self, input_): 
        x = self.encoder(input_)
        x = self.decoder(x)
        return x 
    
    # def __call__(self): 
    #     builtd?  Call: build > call


- tf.keras.models.Model은 Multiinput, Multioutput이 된다.     
-Sequntials를 한방향만 갈 수 있음     
-모델안에 집어 넣을떄는 무조건 Sequential 쓴다 -> "전처리 레이어를 모델의 일부로 사용하기"에서 사용했다.     
-계산복잡도를 줄일 수 있다. 

In [177]:
class MyModel(tf.keras.models.Model): 
    def __int__(self, dim): 
        super(MyModel, self).__init__() 
        self.dim = dim 
        
        # encoder 안에 여러개 동시에 실행해주는 model을 만들어야 함
        #왜? Layers기 때문에 
        self.encoder = tf.keras.models.Sequential([ 
            tf.keras.layers.Flatten(),
            tf.keras.Dense(dim, activation='relu')])  # 연산은 Dense에서 함 
        # Flatten을 시키고, 
    #     self.encoder = tf.keras.layers.Dense(dim)
    
        self.decoder = tf.keras.models.Sequential([ #연산하는것이 아니고, 모양만 바꿔줌-> 그래서 위에 sigmoid 놓기 
            tf.keras.layers.Dense(28*28, activation='sigmoid'),
            tf.keras.layers.Reshape(28,28)
        ]) 
       #self.decoder = tf.keras.layers.Dense(28*28, activation='sigmoid')   #activation은 0~1사이의 값으로 범위결정하는 애.
    def call(self, input_): 
        x = self.encoder(input_)
        x = self.decoder(x)
        return x 
    
  #  print(X_train.shape)
  #  print(X_test.shape)
    # def __call__(self): 
    #     builtd?  Call: build > call

    

In [178]:
autoencoder = MyModel(32)

- 복잡하게 해서 만들경우 -> tf.keras.models.Model 사용하면 됨 
- 상속을 통해 만들면 '객체지향'을 통해 더 유연한 모델만들기가 가능. 

In [179]:
autoencoder.encoder

AttributeError: 'MyModel' object has no attribute 'encoder'

In [186]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [190]:
# 자기 자신 학습 
autoencoder.fit(X_train, X_test, batch_size=32, epochs=20) 

ValueError: Data cardinality is ambiguous:
  x sizes: 60000
  y sizes: 10000
Please provide data which shares the same first dimension.

In [106]:
# 두번쨰
(X_train, _), (X_test, _) = tf.keras.datasets.fashion_mnist.load_data()

In [107]:
X_train[..., tf.newaxis].shape 
tf.expand_dims(X_train, axis=-1)
X_train = X_train.reshape(60000, 28, 28, 1)

In [109]:
X_test = X_test.reshape(10000, 28, 28, 1)

In [110]:
input_ = tf.keras.Input((28,28,1))

In [111]:
x = tf.keras.layers.Conv2D(16,3, padding='same', activation='relu')(input_)
# 중간에 Maxpooling 쓰자 
x = tf.keras.layers.MaxPooling2D(2,2, padding='same')(x) 

x = tf.keras.layers.Conv2D(8,3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2, padding='same')(x) 

x = tf.keras.layers.Conv2D(8,3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2, padding='same')(x) 

x = tf.keras.layers.Conv2D(8,3, padding='same', activation='relu')(x)
x = tf.keras.layers.UpSampling2D(size=(2,2))(x)
x = tf.keras.layers.Conv2D(8,3, padding='same', activation='relu')(x)
x = tf.keras.layers.UpSampling2D(size=(2,2))(x)

x = tf.keras.layers.Conv2D(16,3, padding='same', activation='relu')(x)
# Pooling을 복원시키는 Unpooliing  
x = tf.keras.layers.UpSampling2D(size=(2,2))(x)    #크기를 키운다. 
x = tf.keras.layers.Conv2D(16,3, padding='same', activation='relu')(x)

In [113]:
t = tf.reshape(tf.range(24), (1,2,3,4))  # 4차원 

In [114]:
t.shape

TensorShape([1, 2, 3, 4])

In [115]:
tf.keras.layers.UpSampling2D((1,2))(t) 
# 2,3 -> 2,6 으로 변했음. 크기만 변화시킨다
# 1, , ,4  개수와 색상은 그대로. 

<tf.Tensor: shape=(1, 2, 6, 4), dtype=int32, numpy=
array([[[[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [20, 21, 22, 23]]]])>

In [116]:
tf.keras.layers.UpSampling2D((3,2))(t) 

<tf.Tensor: shape=(1, 6, 6, 4), dtype=int32, numpy=
array([[[[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [20, 21, 22, 23]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [20, 21, 22, 23]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22

In [118]:
t = tf.reshape(tf.range(12), (1,1,3,4))  # 4차원 

In [119]:
t

<tf.Tensor: shape=(1, 1, 3, 4), dtype=int32, numpy=
array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]]])>

In [120]:
t.shape

TensorShape([1, 1, 3, 4])

In [121]:
tf.keras.layers.UpSampling2D((1,2))(t)

<tf.Tensor: shape=(1, 1, 6, 4), dtype=int32, numpy=
array([[[[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]]]])>

In [122]:
tf.keras.layers.UpSampling2D((2,1))(t)  # 위의 (1,2)와 (2,1)의 차이 

<tf.Tensor: shape=(1, 2, 3, 4), dtype=int32, numpy=
array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]]])>